# 03 - Cálculo de Frequência e Intensidade (EPI, EPN, EPF)

Este notebook realiza:

1. Contagem de meses classificados como EPI, EPN e EPF por **ano** e por **estação**.

2. Cálculo da **intensidade média** dos eventos (precipitação média dos meses classificados).

3. Geração de saídas em formato NetCDF para uso nos próximos notebooks.

As análises são feitas tanto **anualmente** quanto **sazonalmente**:

- DJF (Dez-Jan-Fev) – Verão

- MAM (Mar-Abr-Mai) – Outono

- JJA (Jun-Jul-Ago) – Inverno

- SON (Set-Out-Nov) – Primavera


## Importações e parâmetros

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

In [2]:
# Caminhos para os arquivos
base_dir = "dataset/netcdf_data"
arquivo_precip = os.path.join(base_dir, "chirps_mensal_1981_2023.nc")
arquivo_epi = os.path.join(base_dir, "epi.nc")
arquivo_epn = os.path.join(base_dir, "epn.nc")
arquivo_epf = os.path.join(base_dir, "epf.nc")

# Saída
saida_freq_int = os.path.join(base_dir, "freq_intensidade")
os.makedirs(saida_freq_int, exist_ok=True)


## Carregando dataset

In [3]:
# Dados originais
precip = xr.open_dataarray(arquivo_precip)

# Classificações
epi = xr.open_dataarray(arquivo_epi)
epn = xr.open_dataarray(arquivo_epn)
epf = xr.open_dataarray(arquivo_epf)


## Funções auxiliares para frequência e intensidade

In [4]:
def calcular_frequencia(evento):
    """
    Conta o número de meses classificados como True por ano.
    """
    return evento.groupby("time.year").sum(dim="time")

def calcular_intensidade(evento, precip):
    """
    Calcula a precipitação média dos meses classificados (True) por ano.
    """
    # Máscara com precipitação apenas onde o evento é True
    precip_evento = precip.where(evento)
    
    # Média anual
    return precip_evento.groupby("time.year").mean(dim="time")


## Frequência e intensidade anual

In [5]:
freq_epi_anual = calcular_frequencia(epi)
freq_epn_anual = calcular_frequencia(epn)
freq_epf_anual = calcular_frequencia(epf)

int_epi_anual = calcular_intensidade(epi, precip)
int_epn_anual = calcular_intensidade(epn, precip)
int_epf_anual = calcular_intensidade(epf, precip)


## Agrupamento sazonal

Para análises sazonais, usamos labels personalizadas para DJF, MAM, JJA e SON:

In [6]:
# Mapeamento mês -> estação
estacoes = {12: "DJF", 1: "DJF", 2: "DJF",
            3: "MAM", 4: "MAM", 5: "MAM",
            6: "JJA", 7: "JJA", 8: "JJA",
            9: "SON", 10: "SON", 11: "SON"}

def rotular_estacao(time_index):
    return [estacoes[m] for m in time_index.dt.month.values]

# Adiciona coordenada 'season'
epi.coords["season"] = ("time", rotular_estacao(epi.time))
epn.coords["season"] = ("time", rotular_estacao(epn.time))
epf.coords["season"] = ("time", rotular_estacao(epf.time))
precip.coords["season"] = ("time", rotular_estacao(precip.time))


## Frequência e intensidade sazonal

In [7]:
freq_epi_sazonal = epi.groupby("season").sum(dim="time")
freq_epn_sazonal = epn.groupby("season").sum(dim="time")
freq_epf_sazonal = epf.groupby("season").sum(dim="time")

int_epi_sazonal = precip.where(epi).groupby("season").mean(dim="time")
int_epn_sazonal = precip.where(epn).groupby("season").mean(dim="time")
int_epf_sazonal = precip.where(epf).groupby("season").mean(dim="time")


## Savando os resultados

In [8]:
# Frequência anual
freq_epi_anual.to_netcdf(os.path.join(saida_freq_int, "freq_epi_anual.nc"))
freq_epn_anual.to_netcdf(os.path.join(saida_freq_int, "freq_epn_anual.nc"))
freq_epf_anual.to_netcdf(os.path.join(saida_freq_int, "freq_epf_anual.nc"))

# Intensidade anual
int_epi_anual.to_netcdf(os.path.join(saida_freq_int, "int_epi_anual.nc"))
int_epn_anual.to_netcdf(os.path.join(saida_freq_int, "int_epn_anual.nc"))
int_epf_anual.to_netcdf(os.path.join(saida_freq_int, "int_epf_anual.nc"))

# Frequência sazonal
freq_epi_sazonal.to_netcdf(os.path.join(saida_freq_int, "freq_epi_sazonal.nc"))
freq_epn_sazonal.to_netcdf(os.path.join(saida_freq_int, "freq_epn_sazonal.nc"))
freq_epf_sazonal.to_netcdf(os.path.join(saida_freq_int, "freq_epf_sazonal.nc"))

# Intensidade sazonal
int_epi_sazonal.to_netcdf(os.path.join(saida_freq_int, "int_epi_sazonal.nc"))
int_epn_sazonal.to_netcdf(os.path.join(saida_freq_int, "int_epn_sazonal.nc"))
int_epf_sazonal.to_netcdf(os.path.join(saida_freq_int, "int_epf_sazonal.nc"))


- Arquivos gerados contêm frequências (contagem de meses) e intensidades (mm) para cada pixel.

- Resultados estão separados por **anual** e **sazonal**, prontos para análise de tendência.

- Próximo notebook: aplicar **Mann-Kendall** e **Sen’s slope** sobre frequência/intensidade.
